## CUDA介绍：

## 1. Nvidia GPU架构：

<img src="image_github/GPU.png" width="500" height="400">

**硬件：**
>* SP Streaming processor：最基本的处理单元，也称之为CUDA core。
* SMP Streaming Multiprocessor：一个SMP由多个SP以及其他资源组成。

**每个SMP包含的SP数量依据GPU架构而不同，Fermi架构GF100是32个，GF10X是48个，Kepler架构都是192个，Maxwell都是128个。**
<img src="image_github/SM.png" width="400" height="400">

**软件：**
>* thread：线程。
* block：一个block由多个thread组成，同一个block中的threads可以同步，也可以通过shared memory通信。
* grid：一个grid由多个block组成。
* warp：线程束GPU执行程序时的调度单位（每一个SM执行的最小单位），目前cuda的warp的大小为32，**同在一个warp的线程，以不同数据资源执行相同的指令（并行），SIMT（单指令多线程架构）**。

CUDA的设备在实际执行过程中，会以block为单位。把一个个block分配给SM进行运算；而block中的thread又会以warp（线程束）为单位，对thread进行分组计算。目前CUDA的warp大小都是32，也就是说32个thread会被组成一个warp来一起执行。同一个warp中的thread执行的指令是相同的，只是处理的数据不同。**注意：如果线程块内的线程数不能被32整除，意味着最后的一些线程也会当做一个线程束进行并行计算，但是这些inactive线程也会消耗SM的资源。**

<img src="image_github/threads.png" width="400" height="500">

**软硬件对应关系：**

<img src="image_github/soft_hard.png" width="400" height="400">

**总结：**

* 硬件层面core或SP相当于软件层面的线程。
* 硬件层面的SMP相当于软件层面的线程块。
* 硬件层面的GPU（Device）相当于grid。
* 一个线程块内的一个wrap内的线程可以做到并行计算。
* 不同线程块之间的wrap可以做到并行计算。例如：8个线程块，因此，线程并行数量为8 * 32 = 256。

https://blog.csdn.net/qq_41598072/article/details/82877655?depth_1-utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-1&utm_source=distribute.pc_relevant.none-task-blog-BlogCommendFromBaidu-1

## 2. CPU与GPU：

GPU内线程为轻量级线程。
<img src="image_github/CPU_GPU_thread.JPG" width="600" height="600">

<img src="image_github/flow.png" width="500" height="500">

## 3. CUDA：

**一种基于Nvidia GPU的编程语言。**

### 3.1 CUDA 概念：

>* device：GPU
* host：CPU
* kernal：在GPU上运行的函数代码，由一系列线程共同执行相同的代码。

<img src="image_github/kernal.png" width="400" height="400">

## 3.2 CUDA 编程：

### 3.2.1 核函数：

>**一个核函数对应一个grid**，grid底下又有数个block，每个block是一个thread群组。在同一个block中thread可以通过共享内存（shared memory）来通信，同步。而不同block之间的thread是无法通信的。
><img src="image_github/kernal_1.png" width="400" height="200">
<img src="image_github/kernal_2.png" width="400" height="400">

### 3.2.2 global device host 前缀：

> * global：在CPU上调用，在GPU上执行。目前__global__函数必须由CPU调用，并将并行计算任务发射到GPU的任务调用单元。
> * device：只能在GPU上运行。
> * host：由CPU执行并调用的函数（正常的C函数）。

**device前缀定义的函数只能在GPU上执行，所以device修饰的函数里面不能调用一般常见的函数。global前缀，CUDA允许能够在CPU，GPU两个设备上运行，但是也不能运行CPU里常见的函数。**

### 3.2.3 dim3：

> * threadIdx：线程索引。
* blockIdx：块索引。
* blockDim：块维度。
* gridDim：网格维度。

<img src="image_github/block_thread.JPG" width="500" height="500">

### 3.2.4 blocksPerGrid & threadsPerBlock：

>* blocksPerGrid(width, height, 1)：每一个Grid中有width * height * 1 个blocks。
* threadsPerBlock(width, height, 1)：每个block有width * height * 1 个threads。

**blocksPerGrid与threadsPerBlock的分配对于程序的性能起着至关重要的作用。**

例1，下图所示的向量加法：

<img src="image_github/dim3_example.png" width="400" height="400">

下图中，一个block分配给一个SMP会导致效率低。
<img src="image_github/dim3_example_1.png" width="400" height="400">


<img src="image_github/dim3_example_2.png" width="400" height="400">
<img src="image_github/dim3_example_3.png" width="400" height="400">

例2，矩阵加法：

<img src="image_github/example_1.JPG" width="400" height="400">
<img src="image_github/example_2.JPG" width="400" height="400">
<img src="image_github/example_4.JPG" width="500" height="500">
<img src="image_github/example_3.JPG" width="500" height="500">

**总共N * M个元素，每个线程块里有256个线程，所以，threadsPerBlock(14, 14, 1)。一个线程块有256个线程，因此需要N * M/256个线程块。所以，blocksPerGrid(N/14, M/14, 1)。**

**如何确定dim3：**

1. 首先，判断有多少元素。
2. 一个线程块里有多少个线程。
3. 确定线程块尺寸。
4. 确定网格尺寸。

**确定线程块大小的注意事项：**

<img src="image_github/block_size.png" width="500" height="500">

### 3.2.5 GPU内存管理：

* **GPU动态分配与释放内存：**
><img src="image_github/cudaMalloc.png" width="400" height="400">

* **GPU静态分配内存：**
><img src="image_github/cuda_static_memory.png" width="400" height="400">

* **CPU与GPU内存之间数据复制：第一个参数永远代表注意之后的目的地。**
><img src="image_github/cudaMemcpy.png" width="400" height="400">

例如下图所示：
<img src="image_github/cuda_code_example.png" width="600" height="600">